Experiment settings

In [1]:
import simpy
import numpy as np
import pandas as pd
from numpy import random
import math
RANDOM_SEED = 42
### problem paramters ####
order_time_mu = 2
food_prepare_mu = 3

lamb = 0.4
n_kitchen = 3
n_counter = 2
n_customer = 100
SIM_TIME = 24 * 60 #12 hours

random.seed(RANDOM_SEED)

# helper functions, monitoring and pandas table


In [2]:
def round_down(n, decimals=1):
    multiplier = 10 ** decimals
    return abs(math.floor(n*multiplier + 0.5) / multiplier)

counter_total_service_times = 0
counter_total_idle_times = 0
env = simpy.Environment()
# ,"Clock Time","Interarrival Time","Service Time","Wait Time","Queue Length","Total Time In System"
column_names = ["Arrival Time","Queue time","Service Start Time","Food prepare start","Exit system time","Total Wait Time(Queue+Food)","Service Time","Total Time in System"]
result_fifo = np.zeros((n_customer,len(column_names)))
result_tick = np.zeros((n_customer,len(column_names)))


# Simulation setting
## A) FIFO Algorithim

In [3]:
class Counter(object):
#     Counters to take order
    def __init__(self,env,num_counter):
        self.env = env
        self.counter = simpy.Resource(env,num_counter)
        self.counter_waiting = 0
        self.service_start = None
        
    def take_order(self,cus,env,service_start):
        print("%s is placing order at counter %.2f" %(cus,service_start))
        time_taken_to_place_order = max(random.exponential(scale = order_time_mu),1)
        yield self.env.timeout(time_taken_to_place_order)
        print("Order of %s sent to kitchen at %.2f" %(cus, env.now))
        # Record idle counter and add to total count
        
        
    def receive_order(self,cus,env,resource,service_start):
        global counter_total_idle_times
        global counter_total_service_times
        idle_start = env.now
        with resource.kitchen.request() as my_turn:
            yield my_turn
            yield env.process(resource.prepare_food(cus,env))
            service_end = env.now
            print("%s collected the food at %.2f" %(cus, service_end))
            counter_total_idle_times += (service_end-idle_start)
            counter_total_service_times += (service_end-service_start)
            

class Kitchen(object):
    # Kitchen to prepare food
    def __init__(self,env,num_kitchen):
        self.env = env
        self.kitchen = simpy.Resource(env,num_kitchen)
    
    def prepare_food(self,cus,env):
        print("Kitchen is preparing food for %s at %.2f" %(cus, env.now))
        food_prepare_time = max(2,random.exponential(scale = food_prepare_mu))
        yield self.env.timeout(food_prepare_time)
        print("Cooked food for %s at %.2f" %(cus, env.now))

def customer(env, label, queue, kitchen, data):
#     the customer process arrive at the restaurant and request counter to take order
    label = label-1
    arrive_time = env.now
    print("%s entering the queue at %.2f"%(label,arrive_time))
#     data[label,0]=label
    data[label,0]= round_down(arrive_time)
    with queue.counter.request() as my_turn:
        yield my_turn
        service_start = env.now
        data[label,2] = round_down(service_start)
        queue_time = service_start - arrive_time
        data[label,1]= round_down(queue_time)
        # placing order at counter
        yield env.process(queue.take_order(label,env,service_start))
        # waiting order at counter
        prepare_food_start = env.now
        data[label,3] = round_down(prepare_food_start)
        # counter is idle now
        yield env.process(queue.receive_order(label,env,kitchen,service_start))
        # prepare_food_end = round_down(env.now)
        # counter_total_wait_times += round_down(prepare_food_end - prepare_food_start)
        # receive food from counter
        exit_time = env.now
        data[label,4] = round_down(exit_time)

        kitchen_prepare_duration = (exit_time-prepare_food_start)
        # total wait time
        data[label,5] = round_down(kitchen_prepare_duration+queue_time)
        # total service time
        data[label,6] = round_down(exit_time-service_start)
        # total time in system
        data[label,7] = round_down(exit_time-arrive_time)
        yield env.timeout(1)

## Start FIFO Simulation

In [4]:
counter = Counter(env,n_counter)
kitchen = Kitchen(env,n_kitchen)
# Simlating possion process for customer arrival
def customer_arrivals(env,res_counter):
    """Create new *customer* until the sim time reaches 120. with poisson process"""
    for i in range(n_customer):
        yield env.timeout(random.poisson(1/lamb))
        env.process(customer(env, i+1, res_counter, kitchen, result_fifo))

env.process(customer_arrivals(env,counter))
env.run(until=SIM_TIME)

0 entering the queue at 4.00
0 is placing order at counter 4.00
1 entering the queue at 5.00
1 is placing order at counter 5.00
2 entering the queue at 7.00
Order of 0 sent to kitchen at 8.02
Kitchen is preparing food for 0 at 8.02
3 entering the queue at 9.00
Cooked food for 0 at 10.02
0 collected the food at 10.02
4 entering the queue at 11.00
2 is placing order at counter 11.02
Order of 1 sent to kitchen at 12.01
Kitchen is preparing food for 1 at 12.01
Order of 2 sent to kitchen at 12.02
Kitchen is preparing food for 2 at 12.02
5 entering the queue at 13.00
Cooked food for 1 at 14.01
1 collected the food at 14.01
Cooked food for 2 at 14.02
2 collected the food at 14.02
6 entering the queue at 15.00
3 is placing order at counter 15.01
4 is placing order at counter 15.02
7 entering the queue at 16.00
8 entering the queue at 16.00
Order of 4 sent to kitchen at 16.02
Kitchen is preparing food for 4 at 16.02
Order of 3 sent to kitchen at 16.88
Kitchen is preparing food for 3 at 16.88
Co

## FIFO Result & Analysis

In [5]:
# Converting to pandas
np_arr = np.array(result_fifo).reshape(n_customer,-1)
labels = [*range(1,n_customer+1)]
df_fifo=pd.DataFrame(data = np_arr,index=labels,columns=column_names)
# df_fifo
df_fifo=df_fifo.loc[~(df_fifo==0).all(axis=1)]
df_fifo=df_fifo.loc[~(df_fifo==0).all(axis=1)]
total_wait_time = df_fifo.iloc[:,5].sum()
total_service_time = df_fifo.iloc[:,6].sum()

total_time_in_system = df_fifo.iloc[:,7].sum()
print("")

In [6]:
df_fifo.iloc[1:30,:]

# np.subtract(df_fifo.iloc[:,4],df_fifo.iloc[:,3])

,Arrival Time,Queue time,Service Start Time,Food prepare start,Exit system time,Total Wait Time(Queue+Food),Service Time,Total Time in System
2,5.0,0.0,5.0,12.0,14.0,2.0,9.0,9.0
3,7.0,4.0,11.0,12.0,14.0,6.0,3.0,7.0
4,9.0,6.0,15.0,16.9,18.9,8.0,3.9,9.9
5,11.0,4.0,15.0,16.0,19.5,7.5,4.5,8.5
6,13.0,6.9,19.9,20.9,22.9,8.9,3.0,9.9
7,15.0,5.5,20.5,25.3,27.3,7.5,6.8,12.3
8,16.0,7.9,23.9,24.9,26.9,9.9,3.0,10.9
9,16.0,11.9,27.9,28.9,30.9,13.9,3.0,14.9
10,20.0,8.3,28.3,29.3,34.6,13.6,6.3,14.6
11,21.0,10.9,31.9,32.9,45.9,23.9,14.0,24.9


In [7]:
n_served = df_fifo.shape[0]

def display(i):
    print()
    print("%d Customers served"%n_served)
    print("Total Simulation Time=> %.2f Minutes" % SIM_TIME)
    print("Counter Total Idle Time => %.2f Minutes" % (counter_total_idle_times))
    print()
    print("Average Waiting Time => %.2f Minutes" % (total_wait_time / i))
    print("Average Service Time => %.2f Minutes" % (total_service_time / i))
    print("Average Time Spent In System => %.2f Minutes" % (total_time_in_system / i))

display(n_served)


100 Customers served
Total Simulation Time=> 1440.00 Minutes
Counter Total Idle Time => 402.07 Minutes

Average Waiting Time => 45.48 Minutes
Average Service Time => 6.11 Minutes
Average Time Spent In System => 47.56 Minutes
